1. import

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from utils.image_preprocessing import load_images_from_path
from utils.visualization import save_classification_result

2. settings

In [ ]:
model_path = "../saved_models/EfficientNetB0_seed99.keras"
input_path = "../inputs/input1.jpg"  # or "../inputs/inputs_1/"
save_root = "../results/classification"
class_names = ["Bacterial_spotted", "Healthy", "Powdery_mildew", "PMMoV", "TSWV"]

3. model load

In [ ]:
model = load_model(model_path)

4. image load and preprocessing

In [ ]:
images_np, filenames = load_images_from_path(input_path, model_path)
print(f"🔎 처리할 이미지 수: {len(filenames)}")

5. prediction and save

In [ ]:
for idx, img_array in enumerate(images_np):
    fname = filenames[idx]
    input_folder = os.path.basename(input_path) if os.path.isdir(input_path) else None

    # (1) 예측
    preds = model.predict(np.expand_dims(img_array, axis=0))  # (1, num_classes)
    top3_idx = np.argsort(preds[0])[::-1][:3]
    top3_labels = [class_names[i] for i in top3_idx]
    top3_probs = [preds[0][i] for i in top3_idx]

    # (2) 시각화용 원본 복원 (전처리된 이미지는 정규화된 상태)
    restored_img = (img_array - np.min(img_array)) / (np.max(img_array) - np.min(img_array)) * 255
    restored_img = np.clip(restored_img, 0, 255).astype(np.uint8)

    # (3) 저장
    save_classification_result(
        img_array=restored_img,
        top_preds=top3_labels,
        probs=top3_probs,
        save_dir=save_root,
        input_name=fname,
        input_folder=input_folder
    )